In [303]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, log_loss,r2_score
from torchsummary import summary
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import matplotlib.pyplot as plt

In [304]:
df = pd.read_csv('Concrete_Data.csv')
df

,Cement,Blast,Fly,Water,Superplasticizer,Coarse,Fine,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


In [305]:

y = df['Strength'].values
X = df.drop('Strength', axis = 1).values

In [306]:
scaler_X = MinMaxScaler()
scaler_Y = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, 
                                                    random_state=24)
X_scl_trn = scaler_X.fit_transform(X_train) 
X_scl_tst = scaler_X.transform(X_test) 
y_scl_trn = scaler_Y.fit_transform(y_train.reshape(-1,1))
y_scl_tst = scaler_Y.transform(y_test.reshape(-1,1))

In [307]:
X_torch = torch.from_numpy(X_scl_trn)
y_torch = torch.from_numpy(y_scl_trn)

In [308]:
print(X_torch.size())
print(y_torch.size())

torch.Size([721, 8])
torch.Size([721, 1])


In [309]:
torch.manual_seed(24)
model = nn.Sequential(nn.Linear(in_features=X_scl_trn.shape[1], out_features=1))

In [310]:
summary(model, (1,X_scl_trn.shape[1]))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1, 1]               9
Total params: 9
Trainable params: 9
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [311]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.1
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [312]:
for p in model.parameters():
    print(p)

y_pred = model(X_torch.float())
y_pred[:3]

Parameter containing:
tensor([[ 0.1870, -0.0884, -0.3005,  0.0218,  0.3295, -0.1577, -0.1151,  0.2765]],
       requires_grad=True)
Parameter containing:
tensor([-0.0492], requires_grad=True)


tensor([[ 0.0328],
        [-0.2980],
        [-0.1653]], grad_fn=<SliceBackward0>)

In [313]:
print(y_torch.shape)
print(y_pred.shape)

torch.Size([721, 1])
torch.Size([721, 1])


In [314]:
for epoch in np.arange(0,1000):
       # Forward pass: Compute predicted y by passing x to the model
       y_pred_prob = model(X_torch.float())

       # Compute and print loss
       loss = criterion(y_pred_prob, y_torch.float())
       if epoch%100 == 0:
          print('epoch: ', epoch+1,' loss: ', loss.item())

       # Zero gradients, perform a backward pass, and update the weights.
       optimizer.zero_grad()

       # perform a backward pass (backpropagation)
       loss.backward()

       # Update the parameters
       optimizer.step()
#print('epoch: ', epoch+1,' loss: ', loss.item())

epoch:  1  loss:  0.2992650866508484
epoch:  101  loss:  0.021787749603390694
epoch:  201  loss:  0.019807543605566025
epoch:  301  loss:  0.018873723223805428
epoch:  401  loss:  0.01829976588487625
epoch:  501  loss:  0.017902253195643425
epoch:  601  loss:  0.017613237723708153
epoch:  701  loss:  0.017398782074451447
epoch:  801  loss:  0.017238112166523933
epoch:  901  loss:  0.01711706630885601


In [315]:
X_torch_test = torch.from_numpy(X_scl_tst)

### Inferencing on test set
lin_output = model(X_torch_test.float()) # Equivalent predict_proba / predict

y_pred = lin_output.detach().numpy()
# y_pred_prob = 1 / (1 + np.exp(-np_out))

In [316]:
# y_pred_prob = y_pred_prob.reshape(y_scl_tst.shape[0],) 

# y_pred = np.where(y_pred_prob >= 0.5,1,0)

## Test Set Accuracy Score
print(r2_score(y_scl_tst,y_pred))

# log_loss(y_scl_tst, y_pred_prob)

0.582063258798887
